In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Подход 1: Градиентный бустинг в "лоб"

Один из самых универсальных алгоритмов, изученных в нашем курсе, является градиентный бустинг. Он не очень требователен к данным, восстанавливает нелинейные зависимости, и хорошо работает на многих наборах данных, что и обуславливает его популярность. Вполне разумной мыслью будет попробовать именно его в первую очередь.

### 1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
import pandas
features = pandas.read_csv('./features.csv', index_col='match_id')
X = features.ix[:,:-6]
X.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


### 2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [3]:
skips = len(X) - X.count()
skips[skips > 0]

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64

**first_blood_time** - Событие "первая кровь" не успело произойти за первые 5 минут<br/>
**dire_bottle_time** - Приобретение командой *Dire* предмета "bottle" не успело произойти за первые 5 минут

### 3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [4]:
X = X.fillna(0)
X.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


### 4. Какой столбец содержит целевую переменную? Запишите его название.

In [64]:
y = features['radiant_win']
y.head()

match_id
0    1
1    1
2    0
3    0
4    0
Name: radiant_win, dtype: int64

`radiant_win`: 1, если победила команда Radiant, 0 — иначе

### 5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [75]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import time

fold = sklearn.cross_validation.KFold(n=len(X), n_folds=5, random_state=1, shuffle=True)
for estimator in (10,20,30,40):
    clf = sklearn.ensemble.GradientBoostingClassifier(n_estimators=estimator, random_state=1)
    start = time.time()
    res = sklearn.cross_validation.cross_val_score(clf, X=X, y=y, cv=fold, scoring='roc_auc')
    print("Trees = {}, mean = {}, time = {}".format(estimator, res.mean(), time.time()-start))

Trees = 10, mean = 0.6648329228051768, time = 32.77517819404602
Trees = 20, mean = 0.6821140358894263, time = 57.026045083999634
Trees = 30, mean = 0.6896947573844375, time = 89.9125828742981
Trees = 40, mean = 0.6939341884253406, time = 117.50540494918823


Время: **180 секунд** на `scikit-learn-0.17.0` и **90 секунд** на `scikit-learn-0.17.1`<br/>
Качество: **0.69**

# Отчет по подходу 1.

#### 1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

Список столбцов содержащих пропуски и количества этих пропусков:

```
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
```

`first_blood_time` - Событие "первая кровь" не успело произойти за первые 5 минут<br/>
`dire_bottle_time` - Приобретение командой Dire предмета "bottle" не успело произойти за первые 5 минут

#### 2. Как называется столбец, содержащий целевую переменную?

`radiant_win`: 1, если победила команда `Radiant`, 0 — иначе

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

Время: **180 секунд** на `scikit-learn-0.17.0` и **90 секунд** на `scikit-learn-0.17.1`<br/>
Качество: **0.69**

#### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Имеет смысл, т.к. при 40 деревьях качество лучше на 0.004. Для ускорения обучения при росте числа деревьев можно было бы использовать ранее обученный лес, чтобы добавлять в него деревья на каждой итерации цикла. Сейчас лес на каждойитерации цикла создается заново.

# Подход 2: Логистическая регрессия

Линейные методы работают гораздо быстрее композиций деревьев, поэтому кажется разумным воспользоваться именно ими для ускорения анализа данных. Одним из наиболее распространенных методов для классификации является логистическая регрессия.

Важно: не забывайте, что линейные алгоритмы чувствительны к масштабу признаков! Может пригодиться `sklearn.preprocessing.StandardScaler`.

### 1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [7]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing

XX = sklearn.preprocessing.StandardScaler().fit_transform(X)

fold = sklearn.cross_validation.KFold(n=len(XX), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

CPU times: user 23.7 s, sys: 2.89 s, total: 26.6 s
Wall time: 2min 19s
Quality 0.7162096360099013 for {'C': 0.001}
Quality 0.7163550390674078 for {'C': 0.053578947368421052}
Quality 0.7163506196949511 for {'C': 0.1061578947368421}
Quality 0.7163498870640415 for {'C': 0.15873684210526315}
Quality 0.7163493862285406 for {'C': 0.21131578947368421}
Quality 0.7163489138857712 for {'C': 0.26389473684210524}
Quality 0.7163485505626137 for {'C': 0.3164736842105263}
Quality 0.7163482826608616 for {'C': 0.36905263157894735}
Quality 0.7163478790773753 for {'C': 0.42163157894736841}
Quality 0.7163479744886835 for {'C': 0.47421052631578947}
Quality 0.7163479110031911 for {'C': 0.52678947368421047}
Quality 0.7163477436510473 for {'C': 0.57936842105263153}
Quality 0.71634764195224 for {'C': 0.63194736842105259}
Quality 0.7163476154765955 for {'C': 0.68452631578947365}
Quality 0.7163474873787924 for {'C': 0.73710526315789471}
Quality 0.7163474121660685 for {'C': 0.78968421052631577}
Quality 0.71634745

Наилучшее качество получилось **0.716** при **C=0.0535**, оно превосходит качество полученное при градиентном бустинге. Градиентый бустинг хорошо показывает себя на задачах где признаки хорошо сгруппированны в пространстве, т.к. ориентирован на нахождение таких многомерных скоплений объектов. Логистическая регрессия же позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве.

### 2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [30]:
def drop_from(X):
    drops = ["{}{}_hero".format(c, i) for c in 'rd' for i in range(1,6)]
    X2 = X.drop('lobby_type', axis=1).drop(drops, axis=1)
    return X2

X2 = drop_from(X)
X2.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46,182,-80,225,6,3,3,0,-16


In [12]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing

XX2 = sklearn.preprocessing.StandardScaler().fit_transform(X2)

fold = sklearn.cross_validation.KFold(n=len(XX2), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX2, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

CPU times: user 21.3 s, sys: 2.65 s, total: 24 s
Wall time: 2min 1s
Quality 0.7162333103702634 for {'C': 0.001}
Quality 0.7163870888066319 for {'C': 0.053578947368421052}
Quality 0.7163834710885506 for {'C': 0.1061578947368421}
Quality 0.7163825031832755 for {'C': 0.15873684210526315}
Quality 0.7163817068957056 for {'C': 0.21131578947368421}
Quality 0.7163817483953481 for {'C': 0.26389473684210524}
Quality 0.716381486868796 for {'C': 0.3164736842105263}
Quality 0.7163813682791251 for {'C': 0.36905263157894735}
Quality 0.7163812116132653 for {'C': 0.42163157894736841}
Quality 0.7163807900891609 for {'C': 0.47421052631578947}
Quality 0.7163806259273856 for {'C': 0.52678947368421047}
Quality 0.7163805200114796 for {'C': 0.57936842105263153}
Quality 0.7163804977403704 for {'C': 0.63194736842105259}
Quality 0.716380396109538 for {'C': 0.68452631578947365}
Quality 0.7163803770701749 for {'C': 0.73710526315789471}
Quality 0.7163803579802838 for {'C': 0.78968421052631577}
Quality 0.71638041307

Качество классификатора практически не изменилось, это объяснимо тем что линейная модель плохо работает с категориальными данными, без их специальной обработки. Потому исключение таких данных из выборки - не повлияло на качество.

### 3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [13]:
import numpy as np

heroes = np.unique(features[drops].fillna(0))
unique_heroes = len(heroes)
unique_heroes

108

Существует **108** различных идентификтаоров героев.

### 4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [35]:
import numpy as np

def pick_from(X):
    X_pick = np.zeros((X.shape[0], unique_heroes))
    for i, match_id in enumerate(X.index):
        for p in range(1,6):
            X_pick[i, np.where(heroes == X.ix[match_id, 'r%d_hero' % p])] = 1
            X_pick[i, np.where(heroes == X.ix[match_id, 'd%d_hero' % p])] = -1
    return X_pick

X_pick = pick_from(X)
X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [71]:
#X4 = pandas.concat([X2, pandas.DataFrame(X_pick)], axis=1)
X4 = X2.join(pandas.DataFrame(X_pick, index=features.index))
X4.head()
#not np.any(np.isnan(X4)) and np.all(np.isfinite(X4))

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,0,0,0,0,1,0,0,0,0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,0,0,0,0,0,0,0,0,0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0,0,0,0,0,0,0,0,0,0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,0,-1,0,0,0,0,0,0,0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,0,0,0,0,0,0,0,0,0


### 5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [20]:
import sklearn.cross_validation
import sklearn.ensemble
import sklearn.metrics
import sklearn.preprocessing

XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)

fold = sklearn.cross_validation.KFold(n=len(XX4), n_folds=5, random_state=1, shuffle=True)
clf = sklearn.linear_model.LogisticRegression()
grid = {'C': np.linspace(0.001,1,20)}
gs = sklearn.grid_search.GridSearchCV(clf, grid, scoring='roc_auc', cv=fold, n_jobs=8)
%time gs.fit(XX4, y)
for score in gs.grid_scores_:
    print("Quality {} for {}".format(score.mean_validation_score, score.parameters))
print("Best param: ", gs.best_params_)

CPU times: user 47.5 s, sys: 12.5 s, total: 1min
Wall time: 4min 31s
Quality 0.7516014679254464 for {'C': 0.001}
Quality 0.7518686013251092 for {'C': 0.053578947368421052}
Quality 0.7518615151224496 for {'C': 0.1061578947368421}
Quality 0.751859351097132 for {'C': 0.15873684210526315}
Quality 0.7518591596195053 for {'C': 0.21131578947368421}
Quality 0.7518585917440305 for {'C': 0.26389473684210524}
Quality 0.7518578714847706 for {'C': 0.3164736842105263}
Quality 0.7518574097880937 for {'C': 0.36905263157894735}
Quality 0.7518572340234885 for {'C': 0.42163157894736841}
Quality 0.7518570941945373 for {'C': 0.47421052631578947}
Quality 0.7518569862375235 for {'C': 0.52678947368421047}
Quality 0.7518568295170319 for {'C': 0.57936842105263153}
Quality 0.7518567617161451 for {'C': 0.63194736842105259}
Quality 0.751856571017612 for {'C': 0.68452631578947365}
Quality 0.7518564428058737 for {'C': 0.73710526315789471}
Quality 0.7518564173805292 for {'C': 0.78968421052631577}
Quality 0.7518562426

Качество предсказаний классификатора выросло до **0.752**. До текущего момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

### 6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [54]:
features_test = pandas.read_csv('./features_test.csv', index_col='match_id')
X_test = features_test
X_test = X_test.fillna(0)
X2_test = drop_from(X_test)
X4_test = X2_test.join(pandas.DataFrame(pick_from(X_test), index=features_test.index))
X4_test.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,98,99,100,101,102,103,104,105,106,107
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,4,1103,1089,8,0,1,9,3,1183,...,0,0,1,0,0,0,0,0,0,0
7,1430293357,2,556,570,1,0,0,9,4,1194,...,0,0,0,0,0,0,0,0,0,0
10,1430301774,2,751,808,1,0,0,13,2,421,...,0,0,0,0,0,0,0,0,0,1
13,1430323933,3,708,903,1,1,1,11,2,672,...,0,0,0,0,0,0,0,0,-1,0
16,1430331112,4,1259,661,4,0,0,9,5,1703,...,0,0,1,0,0,0,0,0,0,0


In [70]:
XX4 = sklearn.preprocessing.StandardScaler().fit_transform(X4)
logreg = sklearn.linear_model.LogisticRegression(C=gs.best_params_['C'])
logreg.fit(XX4, y)

XX4_test = sklearn.preprocessing.StandardScaler().fit_transform(X4_test)
res = logreg.predict_proba(XX4_test)
print(res.min(), res.max())
print(res)

0.00354240524042 0.99645759476
[[ 0.17544807  0.82455193]
 [ 0.24195397  0.75804603]
 [ 0.81270878  0.18729122]
 ..., 
 [ 0.76834837  0.23165163]
 [ 0.36821547  0.63178453]
 [ 0.57342602  0.42657398]]


Предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой и их среднее арифметическое равно **0.5**. Минимальное значение - **0.0035**, максимальное – **0.9965**

# Отчет по подходу 2

#### 1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Наилучшее качество получилось **0.716** при **C=0.0535**, оно превосходит качество полученное при градиентном бустинге. Градиентый бустинг хорошо показывает себя на задачах где признаки хорошо сгруппированны в пространстве, т.к. ориентирован на нахождение таких многомерных скоплений объектов. Логистическая регрессия же позволяет найти гиперплоскость максимально разделяющую объекты двух классов - она лучше работает когда объекты слабо сгруппированы в пространстве.

#### 2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?

Качество классификатора практически не изменилось, это объяснимо тем что линейная модель плохо работает с категориальными данными, без их специальной обработки. Потому исключение таких данных из выборки - не повлияло на качество.

#### 3. Сколько различных идентификаторов героев существует в данной игре?

Существует **108** различных идентификтаоров героев.

#### 4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Качество предсказаний классификатора выросло до **0.752**. До текущего момента перечисленные в задании категориальные признаки не вносили значительного вклада при построении класификатора ввиду особенностей класификатора. Теперь же эти данные адаптированы для использования классификатором.

#### 5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

Минимальное значение - **0.0035**, максимальное – **0.9965**, среднее арифметическое - **0.5**